<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/OST_to_Online_Archive_Migratorv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os
import tkinter as tk
from tkinter import messagebox

class EmailMigrator:
    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'source_folder_path': 'N/A',
            'destination_store_name': 'N/A',
            'folders_processed': 0,
            'items_moved': 0,
            'items_failed': 0,
            'folder_details': {}
        }

    def setup_logging(self):
        """Setup comprehensive logging for the migration."""
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'ost_migration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)

        file_handler = logging.FileHandler(log_filename)
        file_handler.setLevel(logging.DEBUG)
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logging.getLogger().addHandler(file_handler)

        logging.info(f"Logging to: {log_filename}")

    def find_folder_by_path(self, namespace, folder_path, store_name=None):
        """
        Find an Outlook folder by its full path.
        If store_name is provided, it will search only within that specific store.
        """
        folders = folder_path.split('\\')

        # Determine which stores to search
        stores_to_search = []
        if store_name:
            for store in namespace.Stores:
                if store.DisplayName == store_name:
                    stores_to_search.append(store)
                    break
        else:
            stores_to_search = [store for store in namespace.Stores]

        if not stores_to_search:
            logging.error(f"Store '{store_name}' not found.")
            return None

        for store in stores_to_search:
            try:
                root_folder = store.GetRootFolder()
                # Check if the first part of the path matches the root folder
                if root_folder.Name == folders[0]:
                    current_folder = root_folder
                    for folder_name in folders[1:]:
                        found = False
                        for folder in current_folder.Folders:
                            if folder.Name == folder_name:
                                current_folder = folder
                                found = True
                                break
                        if not found:
                            break
                    else:  # If the loop completed without a break, the full path was found
                        return current_folder
            except Exception as e:
                continue

        # New logic: check the default mailbox separately
        try:
            logging.info("Attempting to find folder in default mailbox.")
            default_root = namespace.GetDefaultFolder(6).Parent # Get the root of the primary mailbox
            current_folder = default_root
            for folder_name in folders:
                found = False
                for folder in current_folder.Folders:
                    if folder.Name == folder_name:
                        current_folder = folder
                        found = True
                        break
                if not found:
                    break
            else:
                return current_folder
        except Exception:
            pass

        return None

    def find_online_archive(self, namespace):
        """Finds the first online archive and returns its root folder."""
        for store in namespace.Stores:
            # Check for the online archive property
            try:
                if store.IsArchive:
                    logging.info(f"Online Archive found: {store.DisplayName}")
                    return store
            except Exception:
                continue
        return None

    def create_folder_if_not_exists(self, parent_folder, folder_name):
        """Creates a new folder in the parent folder if it doesn't already exist."""
        try:
            return parent_folder.Folders[folder_name]
        except Exception:
            try:
                new_folder = parent_folder.Folders.Add(folder_name)
                logging.info(f"Created new folder: {new_folder.FolderPath}")
                return new_folder
            except Exception as e:
                logging.error(f"Failed to create folder '{folder_name}': {e}", exc_info=True)
                return None

    def migrate_folder_contents(self, source_folder, destination_folder):
        """Moves all items from a source folder to a destination folder with throttling."""
        moved_count = 0
        failed_count = 0

        # Throttling configuration
        THROTTLE_LIMIT = 50  # Move 50 emails
        THROTTLE_DELAY = 1   # Then wait for 1 second

        try:
            items_list = list(source_folder.Items)
            for i, item in enumerate(items_list):
                try:
                    item.Move(destination_folder)
                    moved_count += 1
                    self.migration_report['items_moved'] += 1

                    # Apply throttling
                    if (i + 1) % THROTTLE_LIMIT == 0:
                        logging.info(f"Pausing for {THROTTLE_DELAY} seconds to prevent server throttling...")
                        time.sleep(THROTTLE_DELAY)
                except Exception as e:
                    failed_count += 1
                    self.migration_report['items_failed'] += 1
                    logging.warning(f"Failed to move item '{getattr(item, 'Subject', 'N/A')}' from '{source_folder.FolderPath}': {e}")
        except Exception as e:
            logging.error(f"Error accessing items in folder '{source_folder.FolderPath}': {e}", exc_info=True)
            failed_count += len(items_list) if 'items_list' in locals() else 0

        return moved_count, failed_count

    def migrate_folders_recursively(self, source_folder, destination_parent_folder):
        """Recursively migrates subfolders and their contents."""
        self.migration_report['folders_processed'] += 1

        # Create the corresponding folder in the destination
        destination_folder = self.create_folder_if_not_exists(destination_parent_folder, source_folder.Name)
        if not destination_folder:
            logging.error(f"Failed to create destination folder for '{source_folder.Name}'. Skipping contents.")
            self.migration_report['folder_details'][source_folder.FolderPath] = {
                'moved_items': 0, 'failed_items': len(source_folder.Items) if hasattr(source_folder, 'Items') else 0
            }
            return

        # Move all items from the source to the new destination folder
        moved_count, failed_count = self.migrate_folder_contents(source_folder, destination_folder)

        self.migration_report['folder_details'][source_folder.FolderPath] = {
            'moved_items': moved_count,
            'failed_items': failed_count,
            'total_items_in_source': moved_count + failed_count
        }

        logging.info(f"Processed folder '{source_folder.Name}': {moved_count} moved, {failed_count} failed.")

        # Recursively call this function for all subfolders
        for subfolder in source_folder.Folders:
            self.migrate_folders_recursively(subfolder, destination_folder)

    def run_migration(self):
        """Main function to run the migration process."""
        self.migration_report['start_time'] = datetime.now().isoformat()

        # --- Hardcoded Paths ---
        # The script will now look for these specific folders
        SOURCE_FOLDER_PATH = "Inbox\\Trade&BO\\Trade"
        DESTINATION_STORE_NAME = "Online Archive - user@domain.com"

        pythoncom.CoInitialize()
        outlook = None

        try:
            outlook = win32com.client.Dispatch("Outlook.Application")
            namespace = outlook.GetNamespace("MAPI")

            # Step 1: Find the source folder
            logging.info(f"Finding source folder: '{SOURCE_FOLDER_PATH}'")
            source_folder = self.find_folder_by_path(namespace, SOURCE_FOLDER_PATH)
            if not source_folder:
                raise ValueError(f"Source folder '{SOURCE_FOLDER_PATH}' not found in any store.")

            self.migration_report['source_folder_path'] = source_folder.FolderPath

            # Step 2: Find the destination store
            logging.info(f"Finding destination mailbox: '{DESTINATION_STORE_NAME}'")
            target_store = self.find_folder_by_path(namespace, DESTINATION_STORE_NAME)
            if not target_store:
                raise ValueError(f"Destination mailbox '{DESTINATION_STORE_NAME}' not found.")

            self.migration_report['destination_store_name'] = target_store.DisplayName

            # Step 3: Create the full folder path in the destination
            destination_parent_folder = target_store.GetRootFolder()
            source_path_parts = SOURCE_FOLDER_PATH.split('\\')

            for folder_name in source_path_parts:
                destination_parent_folder = self.create_folder_if_not_exists(destination_parent_folder, folder_name)
                if not destination_parent_folder:
                    raise ValueError(f"Failed to create folder path in destination.")

            destination_folder = destination_parent_folder
            self.migration_report['destination_folder_path'] = destination_folder.FolderPath

            print("\n" + "=" * 50)
            print("CONFIRMATION: You are about to MOVE emails from:")
            print(f"Source: {self.migration_report['source_folder_path']}")
            print(f"To Destination: {self.migration_report['destination_folder_path']}")
            print("\nThis action will MOVE items and CANNOT BE UNDONE.")
            print("Please ensure you have backups of your data.")
            print("=" * 50)

            confirm = input("Type 'CONFIRM' to proceed with the migration: ")
            if confirm.strip().upper() != 'CONFIRM':
                logging.warning("Migration cancelled by user.")
                print("Operation cancelled.")
                return False

            # Step 4: Start the migration, moving contents directly
            self.migration_report['folders_processed'] += 1
            moved_count, failed_count = self.migrate_folder_contents(source_folder, destination_folder)

            self.migration_report['folder_details'][source_folder.FolderPath] = {
                'moved_items': moved_count,
                'failed_items': failed_count,
                'total_items_in_source': moved_count + failed_count
            }
            logging.info(f"Processed folder '{source_folder.Name}': {moved_count} moved, {failed_count} failed.")

            # Recursively handle subfolders
            for subfolder in source_folder.Folders:
                self.migrate_folders_recursively(subfolder, destination_folder)

            # Step 5: Finalize and report
            self.migration_report['end_time'] = datetime.now().isoformat()
            self.generate_report()

            if self.migration_report['items_failed'] == 0:
                logging.info("Migration completed successfully!")
                return True
            else:
                logging.warning(f"Migration completed with {self.migration_report['items_failed']} errors. Check the log file for details.")
                return False

        except Exception as e:
            logging.error(f"Critical error during migration: {e}", exc_info=True)
            messagebox.showerror("Migration Error", f"A critical error occurred. Check logs for details: {e}")
            return False
        finally:
            if outlook:
                del outlook
            pythoncom.CoUninitialize()

    def generate_report(self):
        """Generates a comprehensive report of the migration."""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        print("\n=== MIGRATION SUMMARY ===")
        print(f"Source Folder: {self.migration_report['source_folder_path']}")
        print(f"Destination: {self.migration_report['destination_folder_path']}")
        print(f"Folders Processed: {self.migration_report['folders_processed']}")
        print(f"Total Items Moved: {self.migration_report['items_moved']}")
        print(f"Total Items Failed: {self.migration_report['items_failed']}")

        if self.migration_report['items_failed'] > 0:
            print("\n--- Failed Items Summary ---")
            for folder, details in self.migration_report['folder_details'].items():
                if details['failed_items'] > 0:
                    print(f"  - Folder '{folder}': {details['failed_items']} failed items")
            print("Check the log file for specific details on each failed item.")

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("  Outlook Email Migrator (Hardcoded Paths & Throttling)   ")
    print("=" * 60)
    print("This tool will find a specific folder and move its contents")
    print("to a hardcoded destination, preserving the folder structure.")
    print("\n" + "=" * 60)

    migrator = EmailMigrator()
    migrator.run_migration()